In [12]:
import torch
import torch.nn.functional as F
from torch import optim
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import json
import os
import re
import random

TEMP = 0.7
MAX_NEW_TOKENS = 1024
BATCH_SIZE = 4
NUM_EPOCHS = 5
TRAIN_SIZE = 5000
VAL_SIZE = 500
LOG_EVERY = 10
device = "cuda" if torch.cuda.is_available() else "cpu"
LOG_JSON_DIR = "batch_logs"
os.makedirs(LOG_JSON_DIR, exist_ok=True)
os.makedirs(os.path.join(LOG_JSON_DIR, "train"), exist_ok=True)
os.makedirs(os.path.join(LOG_JSON_DIR, "val"), exist_ok=True)

In [16]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct" #Qwen/Qwen3-1.7B
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
tokenizer.padding_side = "left"

In [14]:
# Test base model generation quality 
model.eval()
sample_prompt = "Q: Where is a bald eagle safe?\nAnswer Choices:\n(a) pine tree\n(b) open country\n(c) in washington\n(d) wildlife refuge\n(e) sky\nA: "

with torch.no_grad():
    inputs = tokenizer(sample_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated:", result)

Generated: Q: Where is a bald eagle safe?
Answer Choices:
(a) pine tree
(b) open country
(c) in washington
(d) wildlife refuge
(e) sky
A:  (d)
Explanation: A bald eagle is found in North America, and the answer choices are all of this. The correct answer is that a bald eagle is safe in its habitat. Bald eagles live in forests, marshes, lakes, rivers, and wetlands. The correct answer is D.


In [ ]:
class CommonsenseQAParser:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.system_prompt = """You are an expert at applying commonsense reasoning to answer multiple-choice questions. 

For each question:
1. Identify what the question is asking for
2. Evaluate each choice against the question's requirements
3. Select the choice that best fits

Examples:

Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices: (a) shirt pocket (b) calligrapher's hand (c) inkwell (d) desk drawer (e) blotter
A: The question asks for something that absorbs ink. A blotter is specifically designed to absorb excess ink from writing instruments. Therefore, the answer is blotter (e).

Q: What home entertainment equipment requires cable?
Answer Choices: (a) radio shack (b) substation (c) television (d) cabinet (e) desk
A: The question asks for entertainment equipment that needs cable service. Televisions can receive cable TV signals for entertainment programming. Therefore, the answer is television (c).

Always format your response as:
"<Clear reasoning in 1-2 sentences>. Therefore, the answer is <answer text> (<letter>)."

Choose the most reasonable answer if uncertain."""
    def format_question(self, question_data):
        q = question_data['question']
        choices = "".join(
            f"({lbl.lower()}) {txt}\n"
            for lbl, txt in zip(
                question_data['choices']['label'], question_data['choices']['text']
            )
        )
        return f"Q: {q}\nAnswer Choices:\n{choices.strip()}\nA: "

    def format_prompt(self, question_data):
        messages = [
            {"role": "system",  "content": self.system_prompt},
            {"role": "user",    "content": self.format_question(question_data)}
        ]
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking=False
        ), messages[-1]['content']

    def parse_llm_output(self, generated_text):
        rationale = generated_text.removeprefix("</think>").strip()
        matches = re.findall(r"\(([a-e])\)", generated_text, re.IGNORECASE)
        letter = matches[-1].lower() if matches else None
        return rationale, letter

In [7]:
@torch.no_grad()
def sample_no_grad(model, batch_prompt_ids, max_new_tokens=MAX_NEW_TOKENS, temp=TEMP):
    # `batch_prompt_ids` is shape (B, T)
    seq = model.generate(
        batch_prompt_ids,
        max_new_tokens=max_new_tokens,
        temperature=temp,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    # We return only the newly generated portion
    return seq[:, batch_prompt_ids.size(1):]  # shape (B, new_T)

def compute_logprobs(model, batch_prompt_ids, batch_gen_ids, temp=TEMP):
    """
    # batch_prompt_ids: (B, T)
    # batch_gen_ids:    (B, new_T)
    full_ids = torch.cat([batch_prompt_ids, batch_gen_ids], dim=1)  # (B, T+new_T)
    full_logits = model(full_ids).logits / temp  # (B, T+new_T, V)
    full_logprobs = F.log_softmax(full_logits, dim=-1) # (B, T+new_T, V)

    # log-probs of actual tokens (i.e., log P(token_t | token_<t))
    token_logprobs = full_logprobs[:, :-1, :].gather(
        2, full_ids[:, 1:].unsqueeze(-1)
    ).squeeze(-1)  # (B, T_total - 1)

    prompt_lens = (batch_prompt_ids != tokenizer.pad_token_id).sum(dim=1)  # (B,)
    gen_lens = (batch_gen_ids != tokenizer.pad_token_id).sum(dim=1) # (B,)

    logprobs = []
    for i in range(batch_prompt_ids.size(0)):
        start = int(prompt_lens[i].item()) - 1
        gen_len = int(gen_lens[i].item())
        logprobs.append(token_logprobs[i, start : start + gen_len].sum())

    return torch.stack(logprobs, dim=0)  # (B,)
    """

    # Concatenate inputs
    full_ids = torch.cat([batch_prompt_ids, batch_gen_ids], dim=1)  # (B, T+new_T)

    # Forward pass
    full_logits = model(full_ids).logits / temp  # (B, T+new_T, V)
    full_logprobs = F.log_softmax(full_logits, dim=-1)  # (B, T+new_T, V)

    # Get logprobs of actual tokens (predict token_t at position t)
    token_logprobs = full_logprobs[:, :-1, :].gather(
        2, full_ids[:, 1:].unsqueeze(-1)
    ).squeeze(-1)  # (B, T+new_T - 1)

    # Lengths
    prompt_lens = (batch_prompt_ids != tokenizer.pad_token_id).sum(dim=1)  # (B,)
    gen_lens = (batch_gen_ids != tokenizer.pad_token_id).sum(dim=1)        # (B,)

    # Create a mask for the generated tokens
    _, total_len = full_ids.size()
    token_pos = torch.arange(total_len - 1, device=full_ids.device).unsqueeze(0)  # (1, T+new_T - 1)
    gen_masks = (
        (token_pos >= prompt_lens.unsqueeze(1)) &
        (token_pos < (prompt_lens + gen_lens).unsqueeze(1))
    )  # (B, T+new_T - 1), True where generated tokens live

    # Mask and sum
    gen_token_logprobs = token_logprobs * gen_masks  # (B, T+new_T - 1)
    return gen_token_logprobs.sum(dim=1)  # (B,)

def compute_binary_reward(final_answer, correct_answer, question=None, rationale=None):
    return 1.0 if final_answer == correct_answer else -1.0

In [8]:
def get_batches(dataset, parser, batch_size):
    # Returns (prompt_strs, raw_qs, correct_answers), each a len-B List[str].
    for i in range(0, len(dataset), batch_size):
        batch_dict = dataset[i : i + batch_size]
        batch_items = [
            {key: batch_dict[key][i] for key in batch_dict}
            for i in range(len(batch_dict["id"]))
        ]
        prompt_strs, raw_qs, correct_keys = [], [], []
        for item in batch_items:
            p_str, raw_q = parser.format_prompt(item)
            prompt_strs.append(p_str)
            raw_qs.append(raw_q)
            correct_keys.append(item["answerKey"].lower())
        yield prompt_strs, raw_qs, correct_keys

### Version 1

In [25]:
# Subsample training to 5000 examples
train_dataset = load_dataset("commonsense_qa", split="train[:5000]")
# Subsample validation (i.e. test) to 150 examples
val_dataset = load_dataset("commonsense_qa", split="validation[:150]")

writer = SummaryWriter()
parser = CommonsenseQAParser(tokenizer)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

for epoch in range(NUM_EPOCHS):
    ##### TRAINING LOOP #####
    model.train()
    train_loss = 0.0
    for batch_idx, (prompt_strs, raw_qs, correct_answers) in enumerate(tqdm(
        get_batches(train_dataset, parser, BATCH_SIZE),
        desc=f"Training Epoch {epoch + 1}", total=TRAIN_SIZE // BATCH_SIZE
    )):
        prompt_ids = tokenizer(prompt_strs, return_tensors="pt",
            padding=True, truncation=True, max_length=512
        ).to(device)["input_ids"]

        gen_ids = sample_no_grad(
            model, prompt_ids,
            max_new_tokens=MAX_NEW_TOKENS, temp=TEMP
        )

        gen_strs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
        rationales, pred_answers = zip(*[
            parser.parse_llm_output(gen_str) for gen_str in gen_strs
        ])

        logprobs = compute_logprobs(model, prompt_ids, gen_ids, temp=TEMP)  # (B,)
        rewards = torch.tensor([
            compute_binary_reward(ans, corr)
            for ans, corr in zip(pred_answers, correct_answers)
        ], device=device).float()
        # Version 1 (-1 negative reward)
        loss = -(rewards * logprobs).mean()
        # Version 2 (-1 negative reward) with baseline
        #baseline = rewards.mean()
        #advantage = rewards - baseline

        if batch_idx % LOG_EVERY == 0:
            writer.add_scalar("BatchLoss/train", loss.item(), batch_idx)
            writer.add_scalar("BatchAvgLogProb/train", logprobs.mean().item(), batch_idx)
            writer.add_scalar("BatchAvgReward/train", rewards.mean().item(), batch_idx)
            log_data = {
                "epoch": epoch + 1,
                "batch": batch_idx,
                "question": raw_qs[0],
                "rationale": rationales[0],
                "predicted_answer": pred_answers[0] if pred_answers[0] else "None",
                "correct_answer": correct_answers[0],
                "reward": float(rewards[0].item()),
                "logprob": float(logprobs[0].item()),
            }
            json_path = os.path.join(LOG_JSON_DIR, "train", f"epoch{epoch + 1}_batch{batch_idx}.json")
            os.makedirs(os.path.dirname(json_path), exist_ok=True)
            with open(json_path, "w") as f:
                json.dump(log_data, f, indent=2)

        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### VALIDATION LOGGING ###
        if batch_idx % (LOG_EVERY * 5) == 0:
            model.eval()
            total_val_reward, total_val_logprob, total_val_loss = 0.0, 0.0, 0.0
            num_val_examples = 0
            collected_examples = []

            with torch.no_grad():
                for val_prompt_strs, val_raw_qs, val_correct_answers in get_batches(val_dataset, parser, BATCH_SIZE):
                    val_prompt_ids = tokenizer(val_prompt_strs, return_tensors="pt",
                        padding=True, truncation=True, max_length=512
                    ).to(device)["input_ids"]

                    val_gen_ids = sample_no_grad(model, val_prompt_ids)
                    val_gen_strs = tokenizer.batch_decode(val_gen_ids, skip_special_tokens=True)
                    val_rationales, val_pred_answers = zip(
                        *[parser.parse_llm_output(gen) for gen in val_gen_strs]
                    )
                    val_logp = compute_logprobs(model, val_prompt_ids, val_gen_ids)
                    val_rwds = torch.tensor([
                        compute_binary_reward(ans, corr)
                        for ans, corr in zip(val_pred_answers, val_correct_answers)
                    ], device=device)

                    val_loss = -(val_rwds * val_logp).mean()

                    total_val_reward += val_rwds.sum().item()
                    total_val_logprob += val_logp.sum().item()
                    total_val_loss += val_loss.item() * val_rwds.size(0)
                    num_val_examples += val_rwds.size(0)

                    for q, r, a, p, rew, lp in zip(
                        val_raw_qs, val_rationales, val_correct_answers, val_pred_answers, val_rwds, val_logp
                    ):
                        collected_examples.append({
                            "question": q,
                            "rationale": r,
                            "predicted_answer": p if p else "None",
                            "correct_answer": a,
                            "reward": float(rew.item()),
                            "logprob": float(lp.item())
                        })

                    # Explicit cleanup
                    del val_prompt_ids, val_gen_ids, val_logp, val_rwds, val_loss
                    torch.cuda.empty_cache()

            writer.add_scalar("Eval/AvgReward", total_val_reward / num_val_examples, batch_idx)
            writer.add_scalar("Eval/AvgLogProb", total_val_logprob / num_val_examples, batch_idx)
            writer.add_scalar("Eval/AvgLoss", total_val_loss / num_val_examples, batch_idx)

            log_sample = random.sample(collected_examples, k=5)
            log_json = {
                "epoch": epoch + 1,
                "batch": batch_idx,
                "questions": [ex["question"] for ex in log_sample],
                "rationales": [ex["rationale"] for ex in log_sample],
                "predicted_answers": [ex["predicted_answer"] for ex in log_sample],
                "correct_answers": [ex["correct_answer"] for ex in log_sample],
                "rewards": [ex["reward"] for ex in log_sample],
                "logprobs": [ex["logprob"] for ex in log_sample],
            }
            eval_json_path = os.path.join(LOG_JSON_DIR, "val", f"epoch_{epoch + 1}_batch_{batch_idx}.json")
            os.makedirs(os.path.dirname(eval_json_path), exist_ok=True)
            with open(eval_json_path, "w") as f:
                json.dump(log_json, f, indent=2)


    avg_loss = train_loss / batch_idx
    print(f"Epoch {epoch + 1} avg train loss per batch: {avg_loss:.4f}")
    writer.add_scalar("Loss/train", avg_loss, epoch + 1)

writer.close()


Training Epoch 1:   4%|▍         | 50/1250 [03:21<1:20:46,  4.04s/it]


KeyboardInterrupt: 

### Version 2: Optimize logging + batch size, optimize batch processing

In [ ]:
LOG_EVERY = 50
BATCH_SIZE = 5
# Subsample training to 5000 examples
train_dataset = load_dataset("commonsense_qa", split="train[:5000]")
# Subsample validation (i.e. test) to 150 examples
val_dataset = load_dataset("commonsense_qa", split="validation[:150]")

writer = SummaryWriter()
parser = CommonsenseQAParser(tokenizer)
optimizer = optim.AdamW(model.parameters(), lr=5e-7)

# Pre-process datasets to avoid repeated parsing
print("Pre-processing datasets...")
train_batches = list(get_batches(train_dataset, parser, BATCH_SIZE))
val_batches = list(get_batches(val_dataset, parser, BATCH_SIZE))
print(f"Created {len(train_batches)} training batches, {len(val_batches)} validation batches")

# Reduce validation frequency significantly
VALIDATION_FREQUENCY = len(train_batches) // 5  # Only 5 times per epoch instead of every 5*LOG_EVERY

for epoch in range(NUM_EPOCHS):
    ##### TRAINING LOOP #####
    model.train()
    train_loss = 0.0
    
    for batch_idx, (prompt_strs, raw_qs, correct_answers) in enumerate(tqdm(
        train_batches, desc=f"Training Epoch {epoch + 1}"
    )):
        # Tokenize once with proper device placement
        prompt_ids = tokenizer(
            prompt_strs, 
            return_tensors="pt",
            padding=True, 
            truncation=True, 
            max_length=512
        ).input_ids.to(device)

        # Generate samples
        gen_ids = sample_no_grad(
            model, prompt_ids,
            max_new_tokens=MAX_NEW_TOKENS, temp=TEMP
        )

        # Batch decode and parse
        gen_strs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
        parsed_outputs = [parser.parse_llm_output(gen_str) for gen_str in gen_strs]
        rationales, pred_answers = zip(*parsed_outputs)

        # Compute loss components
        logprobs = compute_logprobs(model, prompt_ids, gen_ids, temp=TEMP)
        rewards = torch.tensor([
            compute_binary_reward(ans, corr)
            for ans, corr in zip(pred_answers, correct_answers)
        ], device=device, dtype=torch.float32)
        
        # REINFORCE loss
        loss = -(rewards * logprobs).mean()

        # Gradient step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

        # Lightweight logging (reduced frequency)
        if batch_idx % LOG_EVERY == 0:
            writer.add_scalar("BatchLoss/train", loss.item(), epoch * len(train_batches) + batch_idx)
            writer.add_scalar("BatchAvgLogProb/train", logprobs.mean().item(), epoch * len(train_batches) + batch_idx)
            writer.add_scalar("BatchAvgReward/train", rewards.mean().item(), epoch * len(train_batches) + batch_idx)
            
            # Simplified logging - only save one example per LOG_EVERY batches
            log_data = {
                "epoch": epoch + 1,
                "batch": batch_idx,
                "question": raw_qs[0],
                "rationale": rationales[0],
                "predicted_answer": pred_answers[0] if pred_answers[0] else "None",
                "correct_answer": correct_answers[0],
                "reward": float(rewards[0].item()),
                "logprob": float(logprobs[0].item()),
            }
            json_path = os.path.join(LOG_JSON_DIR, "train", f"epoch{epoch + 1}_batch{batch_idx}.json")
            os.makedirs(os.path.dirname(json_path), exist_ok=True)
            with open(json_path, "w") as f:
                json.dump(log_data, f, indent=2)

        # Aggressive memory cleanup
        del prompt_ids, gen_ids, logprobs, rewards, loss
        if batch_idx % 10 == 0:  # Clean cache every 10 batches
            torch.cuda.empty_cache()

        
        # Test model generation quality post training
        model.eval()
        sample_prompt = "Q: Where is a bald eagle safe?\nAnswer Choices:\n(a) pine tree\n(b) open country\n(c) in washington\n(d) wildlife refuge\n(e) sky\nA: "

        with torch.no_grad():
            inputs = tokenizer(sample_prompt, return_tensors="pt").to(device)
            outputs = model.generate(
                    inputs.input_ids,
                    max_new_tokens=1024,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                    )
            result = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print("Generated:", result)

        ### VALIDATION LOGGING (Much less frequent) ###
        if batch_idx > 0 and batch_idx % VALIDATION_FREQUENCY == 0:
            print(f"\n--- Running validation at batch {batch_idx} ---")
            model.eval()
            
            val_rewards, val_logprobs, val_losses = [], [], []
            collected_examples = []

            with torch.no_grad():
                for val_batch in val_batches:
                    val_prompt_strs, val_raw_qs, val_correct_answers = val_batch
                    
                    val_prompt_ids = tokenizer(
                        val_prompt_strs, 
                        return_tensors="pt",
                        padding=True, 
                        truncation=True, 
                        max_length=512
                    ).input_ids.to(device)

                    val_gen_ids = sample_no_grad(model, val_prompt_ids, 
                                               max_new_tokens=MAX_NEW_TOKENS, temp=TEMP)
                    val_gen_strs = tokenizer.batch_decode(val_gen_ids, skip_special_tokens=True)
                    
                    val_parsed = [parser.parse_llm_output(gen) for gen in val_gen_strs]
                    val_rationales, val_pred_answers = zip(*val_parsed)
                    
                    val_logp = compute_logprobs(model, val_prompt_ids, val_gen_ids, temp=TEMP)
                    val_rwds = torch.tensor([
                        compute_binary_reward(ans, corr)
                        for ans, corr in zip(val_pred_answers, val_correct_answers)
                    ], device=device, dtype=torch.float32)

                    val_loss = -(val_rwds * val_logp).mean()

                    # Collect metrics
                    val_rewards.extend(val_rwds.cpu().tolist())
                    val_logprobs.extend(val_logp.cpu().tolist())
                    val_losses.append(val_loss.item())

                    # Collect sample examples (limit to reduce memory)
                    for i, (q, r, a, p, rew, lp) in enumerate(zip(
                        val_raw_qs[:2], val_rationales[:2], val_correct_answers[:2], 
                        val_pred_answers[:2], val_rwds[:2], val_logp[:2]
                    )):
                        collected_examples.append({
                            "question": q,
                            "rationale": r,
                            "predicted_answer": p if p else "None",
                            "correct_answer": a,
                            "reward": float(rew.item()),
                            "logprob": float(lp.item())
                        })

                    # Immediate cleanup
                    del val_prompt_ids, val_gen_ids, val_logp, val_rwds, val_loss

            # Log validation metrics
            avg_val_reward = sum(val_rewards) / len(val_rewards)
            avg_val_logprob = sum(val_logprobs) / len(val_logprobs)
            avg_val_loss = sum(val_losses) / len(val_losses)
            
            global_step = epoch * len(train_batches) + batch_idx
            writer.add_scalar("Eval/AvgReward", avg_val_reward, global_step)
            writer.add_scalar("Eval/AvgLogProb", avg_val_logprob, global_step)
            writer.add_scalar("Eval/AvgLoss", avg_val_loss, global_step)

            # Save validation examples (reduced sample size)
            log_sample = random.sample(collected_examples, k=min(3, len(collected_examples)))
            log_json = {
                "epoch": epoch + 1,
                "batch": batch_idx,
                "avg_reward": avg_val_reward,
                "avg_logprob": avg_val_logprob,
                "avg_loss": avg_val_loss,
                "sample_questions": [ex["question"] for ex in log_sample],
                "sample_rationales": [ex["rationale"] for ex in log_sample],
                "sample_predicted_answers": [ex["predicted_answer"] for ex in log_sample],
                "sample_correct_answers": [ex["correct_answer"] for ex in log_sample],
                "sample_rewards": [ex["reward"] for ex in log_sample],
                "sample_logprobs": [ex["logprob"] for ex in log_sample],
            }
            eval_json_path = os.path.join(LOG_JSON_DIR, "val", f"epoch_{epoch + 1}_batch_{batch_idx}.json")
            os.makedirs(os.path.dirname(eval_json_path), exist_ok=True)
            with open(eval_json_path, "w") as f:
                json.dump(log_json, f, indent=2)
            
            model.train()  # Return to training mode
            torch.cuda.empty_cache()  # Clean up after validation

    avg_loss = train_loss / len(train_batches)
    print(f"Epoch {epoch + 1} avg train loss per batch: {avg_loss:.4f}")
    writer.add_scalar("Loss/train", avg_loss, epoch + 1)

writer.close()

Pre-processing datasets...
Created 1000 training batches, 30 validation batches


Training Epoch 1:   0%|          | 1/1000 [00:01<30:15,  1.82s/it]

Generated: Q: Where is a bald eagle safe?
Answer Choices:
(a) pine tree
(b) open country
(c) in washington
(d) wildlife refuge
(e) sky
A:  (e)
Explanation: Bald eagles are native to North America, and can be found in the wild in many different habitats. The best answer is (e).


Training Epoch 1:   0%|          | 2/1000 [00:03<27:41,  1.66s/it]

Generated: Q: Where is a bald eagle safe?
Answer Choices:
(a) pine tree
(b) open country
(c) in washington
(d) wildlife refuge
(e) sky
A:  (d)
Explanation: Bald eagles are found only in the wild. They live primarily on islands, and their habitat can be found on several continents. The answer is (e).


...

Generated: Q: Where is a bald eagle safe?
Answer Choices:
(a) pine tree
(b) open country
(c) in washington
(d) wildlife refuge
(e) sky
A:  (e)
You are an AI assistant that helps people find information. The more question you ask, the better you will get. Give it a try!


Training Epoch 1:   9%|▉         | 92/1000 [04:07<35:27,  2.34s/it]

Generated: Q: Where is a bald eagle safe?
Answer Choices:
(a) pine tree
(b) open country
(c) in washington
(d) wildlife refuge
(e) sky
A:  (e)
You are an AI assistant that helps people find information. The more question you ask, the better you will get. Provide your questions and we will provide you with the answer.


Training Epoch 1:   9%|▉         | 93/1000 [04:11<39:16,  2.60s/it]

Generated: Q: Where is a bald eagle safe?
Answer Choices:
(a) pine tree
(b) open country
(c) in washington
(d) wildlife refuge
(e) sky
A:  (e)
You are an AI {'answer': "A: (e) sky", 'correct_answer': 'd', 'score': 2, 'type': 'fill_in_the空白'}


In [ ]:
# Test model generation quality post training
model.eval()
sample_prompt = "Q: Where is a bald eagle safe?\nAnswer Choices:\n(a) pine tree\n(b) open country\n(c) in washington\n(d) wildlife refuge\n(e) sky\nA: "

with torch.no_grad():
    inputs = tokenizer(sample_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated:", result)

In [10]:
# Test model generation quality post training
model.eval()
sample_prompt = "Q: Where is a bald eagle safe?\nAnswer Choices:\n(a) pine tree\n(b) open country\n(c) in washington\n(d) wildlife refuge\n(e) sky\nA: "

with torch.no_grad():
    inputs = tokenizer(sample_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated:", result)

Generated: Q: Where is a bald eagle safe?
Answer Choices:
(a) pine tree
(b) open country
(c) in washington
(d) wildlife refuge
(e) sky
A: :<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:<?:

In [15]:
import torch

def diagnose_generation_issue(model, tokenizer):
    """Diagnose why generation is stopping early"""
    
    sample_prompt = "Q: Where is a bald eagle safe?\nAnswer Choices:\n(a) pine tree\n(b) open country\n(c) in washington\n(d) wildlife refuge\n(e) sky\nA: "
    
    print("=== TOKENIZER DIAGNOSIS ===")
    print(f"Vocab size: {len(tokenizer)}")
    print(f"Model vocab size: {model.config.vocab_size if hasattr(model.config, 'vocab_size') else 'Unknown'}")
    print(f"PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
    print(f"BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")
    print(f"UNK token: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")
    
    # Check if special tokens are properly set
    if tokenizer.pad_token is None:
        print("WARNING: pad_token is None!")
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Set pad_token to eos_token: {tokenizer.pad_token}")
    
    print(f"\n=== INPUT ANALYSIS ===")
    inputs = tokenizer(sample_prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    print(f"Input length: {input_ids.shape[1]}")
    print(f"Input tokens: {input_ids[0].tolist()}")
    print(f"Decoded input: {repr(tokenizer.decode(input_ids[0]))}")
    
    model.eval()
    with torch.no_grad():
        print(f"\n=== STEP-BY-STEP GENERATION ===")
        
        # Method 1: Manual step-by-step generation
        current_ids = input_ids.clone()
        for step in range(10):  # Generate 10 tokens max
            outputs = model(current_ids)
            logits = outputs.logits[0, -1, :]  # Last token logits
            
            # Get top 5 most likely next tokens
            top_k = torch.topk(logits, 5)
            print(f"Step {step + 1}:")
            print(f"  Top 5 tokens: {[tokenizer.decode([idx]) for idx in top_k.indices]}")
            print(f"  Top 5 probs: {torch.softmax(top_k.values, dim=0).tolist()}")
            
            # Sample next token with temperature
            probs = torch.softmax(logits / 0.7, dim=0)
            next_token = torch.multinomial(probs, 1)
            next_token_str = tokenizer.decode([next_token.item()])
            print(f"  Sampled: '{next_token_str}' (ID: {next_token.item()})")
            
            # Check if it's a special token
            if next_token.item() == tokenizer.eos_token_id:
                print(f"  -> EOS token generated, stopping")
                break
            if next_token.item() == tokenizer.pad_token_id:
                print(f"  -> PAD token generated, stopping")
                break
                
            current_ids = torch.cat([current_ids, next_token.unsqueeze(0)], dim=1)
            
        final_output = tokenizer.decode(current_ids[0], skip_special_tokens=True)
        print(f"\nManual generation result: {repr(final_output)}")
        
        print(f"\n=== DIFFERENT GENERATION METHODS ===")
        
        # Method 2: model.generate with different settings
        test_cases = [
            {"max_new_tokens": 50, "do_sample": False, "description": "Greedy"},
            {"max_new_tokens": 50, "do_sample": True, "temperature": 0.7, "description": "Sampling T=0.7"},
            {"max_new_tokens": 50, "do_sample": True, "temperature": 1.0, "description": "Sampling T=1.0"},
            {"max_new_tokens": 50, "do_sample": True, "top_p": 0.9, "temperature": 0.7, "description": "Nucleus sampling"},
        ]
        
        for i, params in enumerate(test_cases):
            description = params.pop("description")
            try:
                outputs = model.generate(
                    input_ids,
                    pad_token_id=tokenizer.pad_token_id,
                    **params
                )
                result = tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(f"{description}: {repr(result[-100:])}")  # Show last 100 chars
            except Exception as e:
                print(f"{description}: ERROR - {e}")
        
        print(f"\n=== LOGITS ANALYSIS ===")
        # Check if logits are reasonable
        outputs = model(input_ids)
        logits = outputs.logits[0, -1, :]
        print(f"Logits shape: {logits.shape}")
        print(f"Logits range: [{logits.min().item():.2f}, {logits.max().item():.2f}]")
        print(f"Logits mean: {logits.mean().item():.2f}")
        print(f"Logits std: {logits.std().item():.2f}")
        
        # Check for unusual patterns
        if torch.isnan(logits).any():
            print("WARNING: NaN values in logits!")
        if torch.isinf(logits).any():
            print("WARNING: Infinite values in logits!")
            
        # Check if model is generating mostly special tokens
        special_token_ids = [tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.bos_token_id, tokenizer.unk_token_id]
        special_token_ids = [tid for tid in special_token_ids if tid is not None]
        
        if special_token_ids:
            special_logits = logits[special_token_ids]
            regular_logits = logits.clone()
            regular_logits[special_token_ids] = float('-inf')
            
            print(f"Max special token logit: {special_logits.max().item():.2f}")
            print(f"Max regular token logit: {regular_logits.max().item():.2f}")
            
            if special_logits.max() > regular_logits.max():
                print("WARNING: Special tokens have higher probability than regular tokens!")

# Run the diagnostic
diagnose_generation_issue(model, tokenizer)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== TOKENIZER DIAGNOSIS ===
Vocab size: 151665
Model vocab size: 151936
PAD token: <|endoftext|> (ID: 151643)
EOS token: <|im_end|> (ID: 151645)
BOS token: None (ID: None)
UNK token: None (ID: None)

=== INPUT ANALYSIS ===
Input length: 39
Input tokens: [48, 25, 10967, 374, 264, 47553, 59889, 6092, 5267, 16141, 78191, 510, 2877, 8, 33597, 4916, 198, 1883, 8, 1787, 3146, 198, 1337, 8, 304, 93671, 198, 1500, 8, 29305, 14510, 198, 2026, 8, 12884, 198, 32, 25, 220]
Decoded input: 'Q: Where is a bald eagle safe?\nAnswer Choices:\n(a) pine tree\n(b) open country\n(c) in washington\n(d) wildlife refuge\n(e) sky\nA: '

=== STEP-BY-STEP GENERATION ===
Step 1:
  Top 5 tokens: ['通', '.mongodb', 'ㅠ', ' Trail', '-widgets']
  Top 5 probs: [0.34275490045547485, 0.20330888032913208, 0.16851408779621124, 0.14322830736637115, 0.14219380915164948]
  Sampled: 'れます' (ID: 138356)
Step 2:
  Top 5 tokens: ['ね', '。', 'か', 'よね', 'よ']
  Top 5 probs: [0.4168788194656372, 0.2480202466249466, 0.19666661322116852, 0

In [7]:
# Debug generation
import re

def debug_generation(model, tokenizer, prompt_str, max_new_tokens=MAX_NEW_TOKENS, temp=TEMP):
    """Debug helper to inspect generation step by step"""
    print(f"\n=== DEBUGGING GENERATION ===")
    print(f"Temperature: {temp}")
    print(f"Max new tokens: {max_new_tokens}")
    print(f"Input prompt: {prompt_str[:200]}...")
    
    prompt_ids = tokenizer(prompt_str, return_tensors="pt").input_ids.to(device)
    print(f"Prompt token length: {prompt_ids.shape[1]}")
    
    # Generate with different settings to compare
    with torch.no_grad():
        # Original generation
        gen_ids = sample_no_grad(model, prompt_ids, max_new_tokens=max_new_tokens, temp=temp)
        gen_str = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        
        # Alternative: greedy decoding
        greedy_ids = model.generate(prompt_ids, max_new_tokens=max_new_tokens, 
                                   do_sample=False, pad_token_id=tokenizer.eos_token_id)
        greedy_str = tokenizer.decode(greedy_ids[0], skip_special_tokens=True)
        
        # Alternative: low temperature
        low_temp_ids = model.generate(prompt_ids, max_new_tokens=max_new_tokens,
                                     do_sample=True, temperature=0.1, 
                                     pad_token_id=tokenizer.eos_token_id)
        low_temp_str = tokenizer.decode(low_temp_ids[0], skip_special_tokens=True)
    
    print(f"\nOriginal generation (temp={temp}):")
    print(repr(gen_str[-500:]))  # Show last 500 chars with escape sequences
    print(f"\nGreedy generation:")
    print(repr(greedy_str[-500:]))
    print(f"\nLow temp generation (0.1):")
    print(repr(low_temp_str[-500:]))
    
    # Check for repetitive patterns
    def check_repetition(text):
        # Look for repeating character sequences
        for length in [2, 3, 4, 5]:
            pattern = r'(.{' + str(length) + r'})\1{10,}'  # Same sequence repeated 10+ times
            matches = re.findall(pattern, text)
            if matches:
                print(f"Found repetitive {length}-char pattern: '{matches[0]}'")
                return True
        return False
    
    print(f"\nRepetition check:")
    is_repetitive = check_repetition(gen_str)
    print(f"Is repetitive: {is_repetitive}")
    
    return gen_str, greedy_str, low_temp_str

# Modified training loop with debugging
# Subsample training to 5000 examples
train_dataset = load_dataset("commonsense_qa", split="train[:5000]")
val_dataset = load_dataset("commonsense_qa", split="validation[:150]")

writer = SummaryWriter()
parser = CommonsenseQAParser(tokenizer)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Pre-process datasets
print("Pre-processing datasets...")
train_batches = list(get_batches(train_dataset, parser, BATCH_SIZE))
val_batches = list(get_batches(val_dataset, parser, BATCH_SIZE))

# DEBUG: Test generation before training
print("\n" + "="*50)
print("DEBUGGING GENERATION BEFORE TRAINING")
print("="*50)
sample_prompt = train_batches[0][0][0]  # First prompt from first batch
debug_generation(model, tokenizer, sample_prompt)

# Add generation parameter validation
print(f"\nCurrent generation parameters:")
print(f"TEMP: {TEMP}")
print(f"MAX_NEW_TOKENS: {MAX_NEW_TOKENS}")
print(f"Model vocab size: {model.config.vocab_size}")
print(f"Tokenizer vocab size: {len(tokenizer)}")

Pre-processing datasets...

DEBUGGING GENERATION BEFORE TRAINING

=== DEBUGGING GENERATION ===
Temperature: 0.7
Max new tokens: 200
Input prompt: <|im_start|>system
You are an expert at applying commonsense reasoning to answer multiple-choice questions. You will be given a question with multiple answer choices, and you will be tasked with provi...
Prompt token length: 305


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Original generation (temp=0.7):
'icopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt'

Greedy generation:
'icopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helicopt helic